### Import de librerias

In [1]:
import os
from google.cloud import bigquery
import pandas as pd
import db_dtypes
from pandas_gbq import to_gbq

In [2]:
# Establece la ruta al archivo JSON de las credenciales
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "ruta_a_credenciales_json"

### Interactuar con Bigquery

In [3]:
# Construye un cliente para interactuar con BigQuery
client = bigquery.Client()

### Top_tracks

In [4]:
# Ejecuta una consulta 
query_tracks = """
SELECT *
FROM `id_proyecto.dataset.tabla`
"""

In [ ]:
# Ejecuta la consulta y carga el resultado en un DataFrame de Pandas
df_tracks = client.query(query_tracks).to_dataframe()

In [ ]:
df_tracks

#### Transformaciones

##### Cambiar nombres de columnas

In [7]:
df_tracks.rename(columns={
    'Track_Name': 'Nombre_Cancion',
    'Artist': 'Artista',
    'Album': 'Album',
    'Album_Release_Date': 'Fecha_Lanzamiento_Album',
    'Duration_ms': 'Duracion_ms',
    'Popularity': 'Popularidad',
    'Explicit': 'Explicito',
    'External_URL': 'URL_Externo',
    'Preview_URL': 'URL_Previo',
    'Load_Date': 'Fecha_Carga',
    'Album_Image_URL': 'Imagen_album_url'
}, inplace=True)



In [ ]:
df_tracks

##### Extraer el año de lanzamiento del álbum

In [ ]:
df_tracks['Fecha_Lanzamiento_Album'] = pd.to_datetime(df_tracks['Fecha_Lanzamiento_Album'], errors='coerce')
df_tracks['Anio_Lanzamiento'] = df_tracks['Fecha_Lanzamiento_Album'].dt.year
df_tracks

##### Filtrar por duración

In [ ]:
long_songs = df_tracks[df_tracks['Duracion_ms'] > 300000]  # Canciones mayores a 5 minutos
long_songs

##### Crear una columna de popularidad normalizada

In [ ]:
df_tracks['Popularidad_Norm'] = (df_tracks['Popularidad'] - df_tracks['Popularidad'].min()) / (df_tracks['Popularidad'].max() - df_tracks['Popularidad'].min())
df_tracks

##### Agrupar datos

In [ ]:
average_duration = df_tracks.groupby('Artista')['Duracion_ms'].mean().reset_index()
average_duration.rename(columns={'Duracion_ms': 'Duracion_Promedio_ms'}, inplace=True)
average_duration

##### Reemplazar valores faltantes

In [ ]:
df_tracks['Popularidad'].fillna(df_tracks['Popularidad'].mean(), inplace=True)
df_tracks

##### Crear categorías

In [ ]:
bins = [0, 20, 50, 80, 100]
labels = ['Baja', 'Media', 'Alta', 'Muy Alta']
df_tracks['Popularidad_Categorizada'] = pd.cut(df_tracks['Popularidad'], bins=bins, labels=labels)
df_tracks

##### Pasaje hora argentina de carga

In [ ]:
df_tracks.dtypes

In [ ]:
from datetime import timedelta

# Restar 3 horas a la columna `Fecha_Carga`
df_tracks['Fecha_Carga'] = df_tracks['Fecha_Carga'] - timedelta(hours=3)

# Visualiza el DataFrame actualizado
df_tracks

##### Columna de minutos

In [ ]:
df_tracks['Duracion_min'] = df_tracks['Duracion_ms'] / 60000  # Convierte milisegundos a minutos
df_tracks

#### Cargar a Bigquery

In [ ]:
# Cargar el DataFrame a BigQuery, reemplazando si la tabla ya existe
df_tracks.to_gbq(
    destination_table='proyecto_id.dataset.tabla', 
    project_id='proyecto_id', 
    if_exists='replace'  # 'replace' creará la tabla si no existe
)

### Top_artists

In [19]:

query_artists = """
SELECT *
FROM `proyecto_id.dataset.tabla`
"""

In [ ]:
# Ejecuta la consulta y carga el resultado en un DataFrame de Pandas
df_artists = client.query(query_artists).to_dataframe()
df_artists

#### Transformaciones

##### Cambiar nombres de columnas

In [21]:
df_artists.rename(columns={
    'Rank': 'Rango',
    'Artist_Name': 'Nombre_Artista',
    'Genres': 'Generos',
    'Popularity': 'Popularidad',
    'Followers': 'Seguidores',
    'External_URL': 'URL_Externo',
    'Image_URL': 'URL_Imagen',
    'Load_Date': 'Fecha_Carga'
}, inplace=True)


In [ ]:
df_artists.dtypes

##### Contar generos

In [ ]:
df_artists['Numero_Generos'] = df_artists['Generos'].str.split(',').str.len()
df_artists

##### Filtrar artistas populares

In [ ]:
artistas_populares = df_artists[df_artists['Popularidad'] > 50]
artistas_populares

##### Normalizar seguidores

In [ ]:
df_artists['Seguidores_Norm'] = (df_artists['Seguidores'] - df_artists['Seguidores'].min()) / (df_artists['Seguidores'].max() - df_artists['Seguidores'].min())
df_artists

##### Agregar columna de año de carga:

In [ ]:
df_artists['Anio_Carga'] = df_artists['Fecha_Carga'].dt.year
df_artists

##### Pasaje hora argentina de carga

In [ ]:
from datetime import timedelta

# Restar 3 horas a la columna `Fecha_Carga`
df_artists['Fecha_Carga'] = df_artists['Fecha_Carga'] - timedelta(hours=3)

# Visualiza el DataFrame actualizado
df_artists




##### Primer genero

In [ ]:
# Agregar la columna 'Primer_Genero'
df_artists['Primer_Genero'] = df_artists['Generos'].str.split(',').str[0].str.strip()
df_artists

#### Carga en Bigquery

In [ ]:
# Cargar el DataFrame a BigQuery, reemplazando si la tabla ya existe
df_artists.to_gbq(
    destination_table='proyecto_id.dataset.tabla', 
    project_id='proyecto_id', 
    if_exists='replace'  # 'replace' creará la tabla si no existe
)

### Top_song_argentina

In [29]:
# Ejecuta una consulta 
query_top = """
SELECT *
FROM `proyecto_id.dataset.tabla`
"""

In [ ]:
# Ejecuta la consulta y carga el resultado en un DataFrame de Pandas
df_top = client.query(query_top).to_dataframe()
df_top

#### Transformaciones

##### Cambiar nombres de columnas

In [ ]:
df_top.rename(columns={
    'Rank': 'Rango',
    'Song_Name': 'Nombre_Cancion',
    'Artist': 'Artista',
    'Album': 'Album',
    'Release_Date': 'Fecha_Lanzamiento',
    'Popularity': 'Popularidad',
    'Duration_ms': 'Duracion_ms',
    'External_URL': 'URL_Externo',
    'Load_Date': 'Fecha_Carga',
    'Album_Image': 'Imagen_Album',
    'Play_Count': 'Reproducciones',
    
}, inplace=True)
df_top

##### Columna de minutos

In [ ]:
df_top['Duracion_min'] = df_top['Duracion_ms'] / 60000  # Convierte milisegundos a minutos
df_top

##### Pasaje hora argentina de carga

In [ ]:
from datetime import timedelta

# Restar 3 horas a la columna `Fecha_Carga`
df_top['Fecha_Carga'] = df_top['Fecha_Carga'] - timedelta(hours=3)

# Visualiza el DataFrame actualizado
df_top

#### Carga en BigQuery

In [ ]:
# Cargar el DataFrame a BigQuery, reemplazando si la tabla ya existe
df_top.to_gbq(
    destination_table='proyecto_id.dataset.tabla', 
    project_id='proyecto_id', 
    if_exists='replace'  # 'replace' creará la tabla si no existe
)